In [126]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [127]:
if 'google.colab' in str(get_ipython()):
  from google.colab import drive
  drive.mount('/content/drive')
  proj_dir = "/content/drive/MyDrive/ece884_project/"
else:
  proj_dir = "../"

df = pd.read_csv(f"{proj_dir}data_clean/clean.csv")
column_names = df.columns
df = df.to_numpy()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [128]:
import os
import re
gans_saved = os.listdir(f"{proj_dir}saved_models/")
model_number = [int(re.sub("gan", "", x)) for x in gans_saved]
last_model = max(model_number)

In [130]:
codings_size = 98
batch_size = 500

In [140]:
for i in range(1, 8):
  gan_path = f"{proj_dir}saved_models/gan{i}"
  print(gan_path)
  last_gan = tf.saved_model.load(gan_path)
  noise = tf.random.normal(shape=[batch_size, codings_size])
  generated_data = last_gan(noise)
  generated_label = np.concatenate([np.ones((batch_size, )),
                                    np.zeros((batch_size, ))], axis=0)
  idx = np.random.randint(codings_size, size=batch_size)
  real_data = df[idx,:] # sample from full dataset
  generated_and_real = np.concatenate([generated_data,real_data], axis=0)
  shuffeled_indx = np.random.permutation(2*batch_size)
  generated_and_real_shuffeled = generated_and_real[shuffeled_indx, :]
  generated_label_shuffeled = generated_label[shuffeled_indx]

  from sklearn.linear_model import LogisticRegression
  from sklearn.metrics import confusion_matrix

  clf = LogisticRegression(random_state=0)\
    .fit(generated_and_real_shuffeled, generated_label_shuffeled)
  yhat = clf.predict(generated_and_real_shuffeled)
  conf =confusion_matrix(generated_label_shuffeled, yhat)
  print(conf)

/content/drive/MyDrive/ece884_project/saved_models/gan1
[[500   0]
 [  0 500]]
/content/drive/MyDrive/ece884_project/saved_models/gan2
[[500   0]
 [  0 500]]
/content/drive/MyDrive/ece884_project/saved_models/gan3
[[500   0]
 [  0 500]]
/content/drive/MyDrive/ece884_project/saved_models/gan4
[[500   0]
 [  0 500]]
/content/drive/MyDrive/ece884_project/saved_models/gan5
[[500   0]
 [  0 500]]
/content/drive/MyDrive/ece884_project/saved_models/gan6
[[500   0]
 [  0 500]]
/content/drive/MyDrive/ece884_project/saved_models/gan7
[[500   0]
 [  0 500]]


array([[500,   0],
       [  0, 500]])